# Assignment 4
### Last updated: December 20, 2021

### Name: Aseem Sachdeva

### Uniqname: aseemsa

## Instructions

Please turn in:
1. A Jupyter Notebook file. This file should show all of the required work, including code, results, visualizations (if any), and necessary comments to your code. Irrelevant code and results should be deleted prior to submission.

2. An html file showing the preview of the Notebook. To create this file, select File -> Download as > HTML. 

### Before submitting, please select Kernel -> Restart & Run All.

In [20]:
import re
import time
import pickle

from networkx.drawing.nx_pydot import graphviz_layout
from networkx.algorithms import community
from networkx.algorithms.community import modularity
import networkx as nx

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## <font color='red'>Important Notification </font>

<font color='red' size=2>Please **AVOID** using `community` and `modularity` as your variable names. These are imported as preserved names for networkx submodules. Changing their representations would result in autograder failuers.</font>

In [21]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

## Part 1. Wikipedia Network with Communities

## Data description

In this assignment, we are going to analyze the community structure of a network. We will use a Wikipedia based [Map of Science](https://figshare.com/articles/A_Wikipedia_Based_Map_of_Science/11638932) network for our exploration. In this network, each node represents a Wikipedia page in a domain of science, such as natural science or social science. An edge exists between two nodes if the cosine similarity of their page contents reaches a pre-defined threshold.

In [22]:
G = nx.read_gml('assets/MapOfScience.gml', label='id')

Each node in the graph contains the attributes:
- "name:" the title of the article 
- "Class" the science domain 
- "WikipediaUrl:" the Wikipedia URL 

Let's look at some examples: 

In [23]:
list(G.nodes(data=True))[0:5]

[(0,
  {'label': '0',
   'name': 'Accounting',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Accounting'}),
 (1,
  {'label': '1',
   'name': 'Aerospace engineering',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Aerospace_engineering'}),
 (2,
  {'label': '2',
   'name': 'Agricultural engineering',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Agricultural_engineering'}),
 (3,
  {'label': '3',
   'name': 'Agricultural science',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Agricultural_science'}),
 (4,
  {'label': '4',
   'name': 'Agronomy',
   'Class': 'Applied',
   'WikipediaUrl': 'https://en.wikipedia.org/wiki/Agronomy'})]

The edges contain the cosine similarity of the text of the two articles：

In [24]:
list(G.edges(data=True))[0:5]

[(0, 21, {'CosineSimilarity': 0.369447477753246}),
 (0, 50, {'CosineSimilarity': 0.395432741205435}),
 (0, 70, {'CosineSimilarity': 0.388758063740006}),
 (0, 88, {'CosineSimilarity': 0.371542879166867}),
 (0, 516, {'CosineSimilarity': 0.365688238862527})]

### Q1. (1 point) Let's extract the largest connected component from the above graph. In the following questions, we will focus our analysis on this sub-graph. How many nodes are there in this new network?

In [25]:
# Extract the largest connected component of the original dataset
G = G.subgraph(max(nx.connected_components(G), key=len))
N = G.number_of_nodes()  # stores the number of nodes in this graph

# YOUR CODE HERE


In [26]:
#hidden tests for Question 1 are within this cell

### Q2. (2 points) If we think of science domains as communities, how many communities are there in the network and what are their Classes?

In [27]:
list_of_communities = set() # set of unique community labels
N = None           # number of communities in total

# YOUR CODE HERE
list_of_communities = list(G.nodes(data='Class'))
list_of_communities = [d[1] for d in list_of_communities]
list_of_communities = set(list_of_communities)

N = len(list_of_communities)


In [28]:
#hidden tests for Question 2 are within this cell

### Q3. (4 points) How many nodes does each community have?

Hint: you may want to use the [`Counter`](https://docs.python.org/2/library/collections.html#counter-objects) object for this question.

In [29]:
from collections import Counter

list_of_communities = list(G.nodes(data='Class'))
list_of_communities = [d[1] for d in list_of_communities]
dict_num_community = Counter(list_of_communities) # a Counter object with the format {community_name: number_of_nodes}

dict_num_community

# YOUR CODE HERE


Counter({'Applied': 92, 'Formal': 193, 'Natural': 168, 'Social': 224})

In [30]:
#hidden tests for Question 3 are within this cell

## Part 2. Measures of Partition Quality

We discussed 5 ways to measure the quality of a partition: modularity, coverage, performance, separability, and density. 

Modularity, coverage, and performance can be measured using `networkx` functions in the `algorithms.community` module. You can check all the functions provided by a module with the built-in `dir()` function:

```python
from networkx.algorithms import community
dir(community)
>>> ...
 'coverage',
 'modularity',
 'performance',
```

Descriptions of the three measures are provided in the source code,

1. `networkx.algorithms.community.modularity(G, communities, weight='weight')`

2. [`networkx.algorithms.community.quality.coverage(G, partition)`](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.coverage.html#networkx.algorithms.community.quality.coverage)

3. [`networkx.algorithms.community.quality.performance(G, partition)`](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.performance.html#networkx.algorithms.community.quality.performance)





For separability and density, you will implement your own functions. 

Since separability and density first apply a measure to each community and then takes the average over all communities, we provide a helper function `avg_measure(G, communities, measure)` that computes the average value for a given measure over all communities in a graph:

In [31]:
def avg_measure(G, communities, measure):
    """
    Calculate the average value of a given measure across communities in a graph.
    
    Args:
        G - nx graph object; a graph to operate upon.
        communities - list; a collection of sets of nodes that each comprise a community.
        measure - function; calculates a measure for a community in a graph.
    
    Returns:
        (unnamed) - float; the calculated average measure.
    """
    sum_ = 0
    for comm in communities:
        sum_ += measure(G, comm)
    return sum_ / len(communities)

### Q4. (10 points) Let's begin implementing the function to measure the separability for a single community. That is, measure the ratio of intra-community to inter-community edges. 

If there are 0 inter-community edges, the function should assume that the actual number is 1, making the separability value equal to the number of intra-community edges. 

Hint: `G.edges(community)`, where `community` is a set of nodes, returns the edges that are incident to at least one node in `community`. That is, it returns the edges that have at least one endpoint in `community`. 

In [32]:
def separability_one_community(G, community):
    """
    Calculate the separability of a community by finding the ratio of
    intra-community edges to inter-community edges.
    
    Args:
        G - nx graph object; a graph to operate upon.
        community - set; a collection of nodes that comprise a community.
    
    Returns:
        result - float; the separability in a given community.
    """
    result = None # assign separability to it and return
    
    
    com = list(community)
    intra_comm = []
    for n in com:
        neighbors = list(nx.neighbors(G, n))
        for node in neighbors:
            if node in com:
                intra_comm.append((n, node))
    same = [(a,b) for (a,b) in intra_comm if (b,a) in intra_comm]
    intra_community_edges = len(intra_comm) - (len(same) / 2)
    
    inter_comm = []
    for n in com:
        neighbors = list(nx.neighbors(G, n))
        for node in neighbors:
            if node not in com:
                inter_comm.append((n, node))
    same = [(a,b) for (a,b) in inter_comm if (b,a) in inter_comm]
    inter_community_edges = len(inter_comm) - (len(same) / 2)
    
    
    if inter_community_edges == 0:
        result = intra_community_edges
    else: 
        result = intra_community_edges/inter_community_edges

# YOUR CODE HERE

    
    
    return result


In [33]:
test_applied = set([i[0] for i in list(G.nodes(data="Class")) if i[1] == 'Applied'])
separability_one_community(G, test_applied)

0.2882758620689655

In [34]:
#hidden tests for Question 4 are within this cell

Now we can simply use `avg_measure(G, communities, separability_one_community)` to measure the separability of a partition. 

### Q5. (10 points) Let's now implement the function to measure the density of a single community. That is, the fraction of intra-community edges out of all possible edges. 

In [35]:
def density_one_community(G, community):
    """
    Calculate the density of a community by finding the fraction of
    intra-community edges out of all possible edges.
    
    Args:
        G - nx graph object; a graph to operate upon.
        community - set; a collection of nodes that comprise a community.
    
    Returns:
        result - float; the density of a given community.
    """
    
    result = None # assign density to it and return  
    
    
    com = list(community)
    intra_comm = []
    for n in com:
        neighbors = list(nx.neighbors(G, n))
        for node in neighbors:
            if node in com:
                intra_comm.append((n, node))
    same = [(a,b) for (a,b) in intra_comm if (b,a) in intra_comm]
    intra_community_edges = len(intra_comm) - (len(same) / 2)
    
    
    if len(community) == 1:  # If the community has only one node, just return 1 
        return 1
    else:
        result =  intra_community_edges*2/(len(com)*(len(com)-1))
    
    
    
    

    
    # YOUR CODE HERE
    
    
    
    
    

    return result




In [36]:
#hidden tests for Question 5 are within this cell

Now we can simply use `avg_measure(G, communities, density_one_community)` to measure the density of a partition. 

### Q6. (5 points) What is the modularity, coverage, performance, density, and separability of this network using the science domain as a partition?

In [37]:
nodes_applied = (
    node
    for node, data
    in G.nodes(data=True)
    if data.get("Class") == "Applied"
)

nodes_formal = (
    node
    for node, data
    in G.nodes(data=True)
    if data.get("Class") == "Formal"
)


nodes_natural = (
    node
    for node, data
    in G.nodes(data=True)
    if data.get("Class") == "Natural"
)


nodes_social = (
    node
    for node, data
    in G.nodes(data=True)
    if data.get("Class") == "Social"
)



Applied = G.subgraph(nodes_applied)

Formal = G.subgraph(nodes_formal)


Natural = G.subgraph(nodes_natural)

Social = G.subgraph(nodes_social)


communities = [Applied, Formal, Natural, Social]

mod  = community.modularity(G, communities)  # modularity
cov  = community.quality.coverage(G, communities)  # coverage
perf = community.quality.performance(G, communities)  # performance
den  = (density_one_community(G,set((Applied)))+density_one_community(G,set((Formal)))+density_one_community(G,set((Natural)))+
        density_one_community(G,set((Social))))/4  # density
sep  = (separability_one_community(G,set((Applied)))+separability_one_community(G,set((Formal)))+separability_one_community(G,set((Natural)))+
        separability_one_community(G,set((Social))))/4  # separability



In [38]:
print(mod)
print(cov)
print(perf)
print(den)
print(sep)

0.39848031219396246
0.7152063833052018
0.7425423684371532
0.0693753209666948
1.1403277162954395


In [39]:
#hidden tests for Question 6 are within this cell

## Part 3. Community Detection Algorithms

Now let's apply community detection algorithms to the Wikipedia graph. For this part, we will ignore the science domains since we are trying to find communities purely based on the structure of the network. 

We will begin with the Girvan-Newman algorithm and pick the partition that results in the largest modularity, as described in the lecture. Since computing this partition takes a long time, we have precomputed the communities in the notebook `Girvan-Newman.ipynb` and exported the partition to a pickle file. The pickle file is stored as `assets/answer/max_mod_community`. Note that you do not need to run the notebook `Girvan-Newman.ipynb` since we have already done it for you. We only provide it to you as a reference.

### Q7. (1 point) Load the partition from `assets/answer/max_mod_community`. How many communities are there in this partition?

**Hint**:

The partition is stored as a pickle file, which can be imported as the following example:
```python
with open("my_pickle_file_name", 'rb') as f:
    my_data = pickle.load(f)

```

In [40]:
num_max_mod_communities = None  # number of communities in the partition with the largest modularity value

# YOUR CODE HERE
with open("assets/answer/max_mod_community", 'rb') as f:
    my_data = pickle.load(f)
    
num_max_mod_communities = len(my_data)

num_max_mod_communities

35

In [41]:
#hidden tests for Question 7 are within this cell

### Q8. (5 points) What is the modularity, coverage, performance, density, and separability of this partition?

In [42]:


mod  = community.modularity(G, my_data)  # modularity
cov  = community.quality.coverage(G, my_data)  # coverage
perf = community.quality.performance(G, my_data)  # performance
den = 0
sep = 0

for i in my_data:
    den += density_one_community(G,i)
    

for i in my_data:
    sep += separability_one_community(G,i)

den = den/len(my_data)

sep = sep/len(my_data)


# YOUR CODE HERE


In [43]:
#hidden tests for Question 8 are within this cell

### Q9. (12 points) Find a partition of the network with the [label propagation algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.label_propagation.label_propagation_communities.html) and compute the number of communities in the partition and its modularity, coverage, performance, denisty, and separability. This function uses semi-synchronous updating.

In [44]:
communities = nx.algorithms.community.label_propagation.label_propagation_communities(G)

communities = list(communities)

num_community = len(communities) # number of communities in the partition

den = 0
sep = 0


mod  = community.modularity(G, communities)  # modularity
cov  = community.quality.coverage(G, communities)  # coverage
perf = community.quality.performance(G, communities)  # performance


for i in communities:
    den += density_one_community(G,i)
    

for i in communities:
    sep += separability_one_community(G,i)
    
den = den/num_community

sep = sep/num_community

# YOUR CODE HERE


In [45]:
#hidden tests for Question 9 are within this cell

The [Clauset-Newman-Moore greedy modularity maximization algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.modularity_max.greedy_modularity_communities.html#networkx-algorithms-community-modularity-max-greedy-modularity-communities) implements an Agglomerative Hierarchical Clustering procedure to find a partition with high modularity. 

Note: the function `greedy_modularity_communities` returns a list of `Frozensets`, where each `Frozenset` is simply an immutable Python `set` object (i.e. the elements cannot be modified).



### Q10. (12 points) Using Clauset-Newman-Moore greedy modularity maximization, find a partition of the network and compute the number of communities in the partition and its modularity, coverage, performance, density, and separability.

In [46]:
communities = nx.algorithms.community.greedy_modularity_communities(G)

communities = list(communities)

num_community = len(communities) # number of communities in the partition

den = 0
sep = 0


mod  = community.modularity(G, communities)  # modularity
cov  = community.quality.coverage(G, communities)  # coverage
perf = community.quality.performance(G, communities)  # performance


for i in communities:
    den += density_one_community(G,i)
    

for i in communities:
    sep += separability_one_community(G,i)
    
den = den/num_community

sep = sep/num_community

# YOUR CODE HERE





In [47]:
#hidden tests for Question 10 are within this cell

### Q11. (6 points) Using [asynchronous Fluid Communities algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.asyn_fluid.asyn_fluidc.html#networkx.algorithms.community.asyn_fluid.asyn_fluidc) with parameters  `max_iter` = 100 and `seed` = 233 (see Tutorial for details), find the parameter `k` between `k`=3 and `k`=40 that maximizes the modularity of the partition. Note that `k` represents the number of communities in the partition. 

In [48]:
best_k = None #optimal value of k

mod_k_list = []

for i in range(3, 41):

    fluid_communities = list(nx.algorithms.community.asyn_fluidc(G, k=i, max_iter=100, seed=233)) #partition returned by the algorithm with k = best_k
    mod  = community.modularity(G, fluid_communities)  # modularity
    mod_k_list.append((mod, i)) 

mod_k_list = sorted(mod_k_list,key=lambda x: x[0], reverse=True)

best_k = mod_k_list[0][1]    
fluid_communities = list(nx.algorithms.community.asyn_fluidc(G, k=11, max_iter=100, seed=233))    
# YOUR CODE HERE


In [49]:
#hidden tests for Question 11 are within this cell

### Q12. (6 points) Using [asynchronous Fluid Communities algorithm](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.asyn_fluid.asyn_fluidc.html#networkx.algorithms.community.asyn_fluid.asyn_fluidc) with parameters  `max_iter` = 100, `seed` = 233, and the value of `k` you found in the previous question, compute the number of communities in the partition and its modularity, coverage, performance, density, and separability.

In [50]:

fluid_communities = list(nx.algorithms.community.asyn_fluidc(G, k=11, max_iter=100, seed=233))    


num_community = len(fluid_communities) # number of communities in the partition

mod  = community.modularity(G, fluid_communities)  # modularity
cov  = community.quality.coverage(G, fluid_communities)  # coverage
perf = community.quality.performance(G, fluid_communities)  # performance

den = 0
sep = 0


for i in fluid_communities:
    den += density_one_community(G,i)
    

for i in fluid_communities:
    sep += separability_one_community(G,i)
    
den = den/num_community

sep = sep/num_community

# YOUR CODE HERE








In [51]:
#hidden tests for Question 12 are within this cell

In [52]:
num_community

11

### Q13. (6 points) Create and print a pandas DataFrame where each row represents a community detection algorithm and each column is a quality measure, as described by the following scheme:


|.|Method name|num_community|modularity|coverage|performance|density|separability|
|-|-|-|-|-|-|-|-|
|0|Girvan–Newman|
|1|Greedy modularity maximization|
|2|Fluid Communities|
|3|Label propogation|


In [53]:
compare = None # assign your pandas dataframe here







In [54]:
# YOUR CODE HERE

##Girvan-newman
with open("assets/answer/max_mod_community", 'rb') as f:
    my_data = pickle.load(f)
    
num_community_girvan = len(my_data)

mod_girvan  = community.modularity(G, my_data)  # modularity
cov_girvan  = community.quality.coverage(G, my_data)  # coverage
perf_girvan = community.quality.performance(G, my_data)  # performance
den_girvan = 0
sep_girvan = 0
for i in my_data:
    den += density_one_community(G,i)
for i in my_data:
    sep += separability_one_community(G,i)
den_girvan = den/num_community_girvan
sep_girvan = sep/num_community_girvan


In [55]:
#Greedy modularity
communities = nx.algorithms.community.greedy_modularity_communities(G)

communities = list(communities)

num_community_greedy = len(communities) # number of communities in the partition

den_greedy = 0
sep_greedy = 0


mod_greedy  = community.modularity(G, communities)  # modularity
cov_greedy  = community.quality.coverage(G, communities)  # coverage
perf_greedy = community.quality.performance(G, communities)  # performance


for i in communities:
    den += density_one_community(G,i)
    

for i in communities:
    sep += separability_one_community(G,i)
    
den_greedy = den/num_community_greedy

sep_greedy = sep/num_community_greedy

# YOUR CODE HERE



In [56]:
#Fluid communities

fluid_communities = list(nx.algorithms.community.asyn_fluidc(G, k=11, max_iter=100, seed=233))    


num_community_fluid = len(fluid_communities) # number of communities in the partition

mod_fluid  = community.modularity(G, fluid_communities)  # modularity
cov_fluid  = community.quality.coverage(G, fluid_communities)  # coverage
perf_fluid = community.quality.performance(G, fluid_communities)  # performance

den_fluid = 0
sep_fluid = 0


for i in fluid_communities:
    den_fluid += density_one_community(G,i)
    

for i in fluid_communities:
    sep_fluid += separability_one_community(G,i)
    
den_fluid = den/num_community_fluid

sep_fluid = sep/num_community_fluid

# YOUR CODE HERE



In [57]:
#Label Propogation
label_prop_communities = list(nx.algorithms.community.label_propagation.label_propagation_communities(G))

num_community_label = len(label_prop_communities)

mod_label  = community.modularity(G, label_prop_communities)  # modularity
cov_label  = community.quality.coverage(G, label_prop_communities)  # coverage
perf_label = community.quality.performance(G, label_prop_communities)  # performance

den_label = 0
sep_label = 0


for i in label_prop_communities:
    den_label += density_one_community(G,i)
    

for i in label_prop_communities:
    sep_label += separability_one_community(G,i)
    
den_label = den/num_community_label

sep_label = sep/num_community_label

# YOUR CODE HERE


In [58]:
compare = pd.DataFrame()

method_name = ['Girvan–Newman', 'Greedy modularity maximization', 'Fluid Communities', 'Label propogation']
num_community = [num_community_girvan, num_community_greedy, num_community_fluid, num_community_label]
modularity = [mod_girvan, mod_greedy, mod_fluid, mod_label]
coverage = [cov_girvan, cov_greedy, cov_fluid, cov_label]
performance = [perf_girvan, perf_greedy, perf_fluid, perf_label]
density = [den_girvan, den_greedy, den_fluid, den_label]
separability = [sep_girvan, sep_greedy, sep_fluid, sep_label]

compare['Method name'] = method_name
compare['num_community'] = num_community
compare['modularity'] = modularity
compare['coverage'] = coverage
compare['performance'] = performance
compare['density'] = density
compare['separability'] = separability



In [59]:
compare

,Method name,num_community,modularity,coverage,performance,density,separability
0,Girvan–Newman,35,0.580687,0.815252,0.888706,0.660259,1.118761
1,Greedy modularity maximization,14,0.550356,0.805892,0.775095,2.259167,4.277759
2,Fluid Communities,11,0.590856,0.728249,0.915128,2.875303,5.444420
3,Label propogation,28,0.584188,0.808808,0.857888,1.129583,2.138879


In [60]:
#hidden tests for Question 13 are within this cell

### Q14. (10 points) Does it appear that an algorithm consistently performs better than the others across the different quality measures? Explain. 

You do not need to explain your answer based on the details of the algorithms or quality measures. Do so only based on the quality scores on the dataframe in Q13. 

You should not consider the number of communities to answer this question. 



Of all four algorithms explored above, the Fluid Communities algorithm appears to perform the best across most of the quality measures. The algorithm has the highest modularity score, which implies that it performs the best of all algorithms in regards to dividing nodes into distinct modules(dense connections between nodes within the module but sparse connections between nodes cross different modules). It has the worst covrage score(Girvan-Newman has the highest), which implies that its ratio of intra-community edges(edges joining nodes in the same partition) against all nodes in the network is the lowest. It has the highest performance score, which boils down to intra-community edges plus inter-community edges(edges joining nodes in different blocks of the partition) divided by total edges. It has the highest density score, which implies that nodes in the network connects to the largest amount of "potential" connections compared to the other algorithms(calculated by measuring the fraction of intra-community edges out of all possible edges and averaging over all communities). Finally, the Fluid Communities algorithm boasts the highest sperability score, which is the ratio of intra-community edges to inter-community edges for each community, averaged over all communities. All of these measures are just different ways to assess the quality of partitions in a network, and the Fluid communities algorithm does well on the majority.

### Q15. (10 points) Comparing the quality of the partition based on the science domain with that of the partitions generated by the various algorithms, from a network structure perspective, does the science domain appear to be a good way to partition the network into communities? Explain.

Recall that you already computed the quality of the partition based on science domain in Q6. Be sure to use those results when answering this question. 

You should not consider the number of communities to determine if a partition is "good." For this question, focus on the quality scores. 

Computing the quality of the partition based on the science domain yeilds a modularity, coverage, performance, density, and separability score of 0.398, 0.715, 0.742, 0.069, and 1.14 respectively. This methodology produces the lowest modularity, coverage, performance, and density scores, and the fourth lowest separability score(only higher than Girvan-Newman). It summarily does not appear the most viable method for paritioning the network. It is possible that there is a fair amount of intersectionality between science domains, in the sense that its not unreasonable to assume that concepts overlap. Therefore, there may be a fair amount of edges that lie between communities using this method. 

# End